In [1]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [3]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

# temp = AssistantAgent("aa", model_client=model, system_message="")

clarity_agent = AssistantAgent(
    name = "ClarityAgent",
    model_client=model,
    system_message= """
                    You are an expert editor focused on clarity and simplicity.
                    Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear.
                    Don't worry about persuasion or tone — just make the message easy to read and understand.
                    """
)

tone_agent  = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message= """
                    You are a communication coach focused on emotional tone and professionalism.
                    Your job is to make the email sound warm, confident, and human — while staying professional
                    and appropriate for the audience. Improve the emotional resonance, polish the phrasing,
                    and adjust any words that may come off as stiff, cold, or overly casual.
                    """
)

persuasion_agent  = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message= """
                    You are a persuasion expert trained in marketing, behavioral psychology, and copywriting.
                    Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. 
                    Remove weak or passive language.
                    """
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message= """
                    You are an advanced email-writing specialist. Your role is to read all
                    prior agent responses and revisions, and then **synthesize the best ideas** into a unified,
                    polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements;
                    Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional,
                    effective, and readable.
                    """
)

critic_agent  = AssistantAgent(
    "criticAgent",
    model_client=model,
    system_message= """
                    You are an email quality evaluator. Your job is to perform a final review
                    of the synthesized email and determine if it meets professional standards. Review the email for:
                    Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
                    Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone,
                    or missing key elements), provide ONE specific improvement suggestion. If the email meets professional standards and communicates effectively, respond with 'The email meets professional standards.' followed by `TERMINATE` on a new line. You should only approve emails that are perfect enough for professional use, dont settle.
                    """
)

In [5]:
text_termination = TextMentionTermination("TERMINATE")
max_message_termination = MaxMessageTermination(max_messages=30)

termination_conditions = text_termination | max_message_termination

team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent
    ],
    termination_condition=termination_conditions
)

await Console(
    team.run_stream(task="안녕! 나 배고파 점심사줘~")
)

---------- TextMessage (user) ----------
안녕! 나 배고파 점심사줘~
---------- TextMessage (ClarityAgent) ----------
안녕! 나 배고파. 점심 사줘.
---------- TextMessage (ToneAgent) ----------
제목: 점심 부탁드려요

안녕하세요 [받는 분의 이름],

잘 지내고 계신가요? 오늘 조금 배고파서 점심을 사주실 수 있을지 여쭤보고 싶어요. 정말 감사할 것 같아요!

늘 고맙습니다!

좋은 하루 되세요,  
[내 이름]
---------- TextMessage (PersuasionAgent) ----------
제목: 점심 한끼 부탁드립니다!

안녕하세요 [받는 분의 이름],

잘 지내고 계신가요? 오늘 배가 고파서 혹시 점심을 한 끼 함께할 수 있을까요? 당신과 함께하는 점심 시간이 정말 즐거울 것 같아 기대가 됩니다!

좋아하는 식당에서 맛있는 음식을 누리며, 좋은 대화도 나눌 수 있는 기회가 되면 좋겠네요. 물론, 이 은혜는 꼭 갚겠습니다!

부탁드리며, 감사합니다! 좋은 하루 되세요!

감사합니다,  
[내 이름]
---------- TextMessage (SynthesizerAgent) ----------
제목: 점심 한 끼 부탁드립니다!

안녕하세요 [받는 분의 이름],

잘 지내고 계신가요? 오늘 배가 고파서 혹시 점심을 한 끼 함께할 수 있을지 여쭤보고 싶습니다. 당신과 함께하는 점심 시간이 정말 기대되네요!

좋아하는 식당에서 맛있는 음식을 나누며 좋은 대화를 나눌 수 있었으면 좋겠습니다. 그리고 물론, 이 은혜는 꼭 갚겠습니다!

부탁드리며, 미리 감사드립니다! 좋은 하루 되세요!

감사합니다,  
[내 이름]
---------- TextMessage (criticAgent) ----------
The email meets professional standards.  
TERMINATE


TaskResult(messages=[TextMessage(id='e3e564e4-2351-4708-ba85-de43ebc59ca9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 30, 7, 19, 39, 283660, tzinfo=datetime.timezone.utc), content='안녕! 나 배고파 점심사줘~', type='TextMessage'), TextMessage(id='a8143ae3-7496-4d9e-8638-d37d4073b5cc', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=103, completion_tokens=13), metadata={}, created_at=datetime.datetime(2025, 9, 30, 7, 19, 39, 912107, tzinfo=datetime.timezone.utc), content='안녕! 나 배고파. 점심 사줘.', type='TextMessage'), TextMessage(id='825d85d5-b909-4d60-822e-213fc8d738a6', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=207, completion_tokens=75), metadata={}, created_at=datetime.datetime(2025, 9, 30, 7, 19, 41, 564572, tzinfo=datetime.timezone.utc), content='제목: 점심 부탁드려요\n\n안녕하세요 [받는 분의 이름],\n\n잘 지내고 계신가요? 오늘 조금 배고파서 점심을 사주실 수 있을지 여쭤보고 싶어요. 정말 감사할 것 같아요!\n\n늘 고맙습니다!\n\n좋은 하루 되세요,  \n[내 이름]', type='TextMessage'), TextMessage(id='7b7fcb